# Artik 47

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14530019185048515595
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2255906407
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10831226224613394344
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [14]:
gbpusd = pd.read_csv(f'~\Documents\Artik_47\Artik-47\data\GBPUSD_M30.csv', sep='\t')
gbpusd

,Time,Open,High,Low,Close,Volume
0,2007-01-01 06:00:00,1.95805,1.95836,1.95795,1.95817,1728
1,2007-01-01 06:30:00,1.95813,1.95835,1.95788,1.95815,1651
2,2007-01-01 07:00:00,1.95815,1.95832,1.95807,1.95824,1352
3,2007-01-01 07:30:00,1.95818,1.95833,1.95807,1.95822,1722
4,2007-01-01 08:00:00,1.95815,1.95871,1.95808,1.95852,1729
...,...,...,...,...,...,...
193895,2022-07-13 03:30:00,1.18882,1.18885,1.18852,1.18863,596
193896,2022-07-13 04:00:00,1.18858,1.18894,1.18796,1.18813,1522
193897,2022-07-13 04:30:00,1.18811,1.18827,1.18779,1.18794,1852
193898,2022-07-13 05:00:00,1.18793,1.18814,1.18744,1.18797,1404
